In [1]:
from openai import OpenAI
client = OpenAI()

client.files.create(
  file=open("test_reworded_questions_data.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-VjtQ25HUXzgJ7LyWTiZMGB', bytes=27905, created_at=1755739368, filename='test_reworded_questions_data.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

In [ ]:
client.fine_tuning.jobs.create(
  training_file="file-abc123",
  model="gpt-4.1-mini-2025-04-14"
)